<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## Phonemizer

Assumes espeak backend is installed via `apt-get install espeak`

In [1]:
#| echo: false
#| output: asis
show_doc(Phonemizer)

---

[source](https://github.com/slegroux/nimrod/blob/main/nimrod/text/tokenizers.py#L16){target="_blank" style="float:right; font-size:smaller"}

### Phonemizer

>      Phonemizer (separator=<phonemizer.separator.Separator object at
>                  0x7fb6d88c8370>, language='en-us', backend='espeak',
>                  strip=True, preserve_punctuation=True)

Initialize self.  See help(type(self)) for accurate signature.

## Usage

In [ ]:
p = Phonemizer()
text = "Oh Dear! This suck...\n We'll be fine!"
print(p(text))

oʊ dɪɹ! ðɪs sʌk...
wiːl biː faɪn!


## Tokenizer

Requires download of spacy specific language e.g. `python -m spacy download en`

In [2]:
#| echo: false
#| output: asis
show_doc(Tokenizer)

---

[source](https://github.com/slegroux/nimrod/blob/main/nimrod/text/tokenizers.py#L54){target="_blank" style="float:right; font-size:smaller"}

### Tokenizer

>      Tokenizer (backend='spacy', language='en')

Initialize self.  See help(type(self)) for accurate signature.

## Numericalizer

In [3]:
#| echo: false
#| output: asis
show_doc(Numericalizer)

---

[source](https://github.com/slegroux/nimrod/blob/main/nimrod/text/tokenizers.py#L69){target="_blank" style="float:right; font-size:smaller"}

### Numericalizer

>      Numericalizer (tokenizer:__main__.Tokenizer)

Initialize self.  See help(type(self)) for accurate signature.

In [ ]:
# TODO: collate text + add special characters & 0 != unk

## Usage

In [ ]:
tok = Tokenizer()
tokenized = tok("Oh, yeah\n I don't know dude...")
ds = AG_NEWS(split='test') # data pipe
sample = next(iter(ds)) # (label, text)
print(sample)
tokenized_ds = tok.tokenize_iter(ds)
sample = next(iter(tokenized_ds))
print(sample)

/home/syl20/anaconda3/envs/nimrod/lib/python3.9/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


(3, "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.")
['Fears', 'for', 'T', 'N', 'pension', 'after', 'talks', 'Unions', 'representing', 'workers', 'at', 'Turner', '  ', 'Newall', 'say', 'they', 'are', "'", 'disappointed', "'", 'after', 'talks', 'with', 'stricken', 'parent', 'firm', 'Federal', 'Mogul', '.']


In [ ]:
num = Numericalizer(tok)
mapper = num.build_map_from_iter(ds)
print(mapper["<unk>"])
print(mapper(tok("here we go. asdflkj")))
# text_pipeline = lambda x: voc(tokenizer(x))
print(mapper(tokenized))

0
[531, 1037, 307, 3, 0]
[7808, 2, 0, 0, 296, 378, 255, 1324, 0, 64]


In [ ]:
a = mapper(tok("here we go. asdflkj"))
print(len(a))
b = mapper(tok("Oh, yeah\n I don't know dude..."))
print(len(b))
mini_batch = [a, b]
x = [torch.LongTensor(x_i) for x_i in mini_batch]
print(x)
x_padded = pad_sequence(x, batch_first=True, padding_value=0)
print(x_padded)

5
10
[tensor([ 531, 1037,  307,    3,    0]), tensor([7808,    2,    0,    0,  296,  378,  255, 1324,    0,   64])]
tensor([[ 531, 1037,  307,    3,    0,    0,    0,    0,    0,    0],
        [7808,    2,    0,    0,  296,  378,  255, 1324,    0,   64]])


In [ ]:
def text_collate(batch):
    # batch: [(label, text), ]
    # from ipdb import set_trace; set_trace()
    texts = [row[1] for row in batch]
    tokens = [torch.LongTensor(mapper(tok(row))) for row in texts]
    text_lens = [len(token) for token in tokens]
    text_pad = pad_sequence(tokens, batch_first=True, padding_value=-1)
    return text_pad, text_lens

In [ ]:
dl = DataLoader(dataset=ds, batch_size=5, shuffle=True, collate_fn=text_collate)

In [ ]:
b = next(iter(dl))
print(b)

(tensor([[11991, 29434, 24121, 10187,  2896,   147,     6,   238,    17,    28,
            81,  4305,    20,    12,    13, 16249,    13,    76,    27,    26,
             1,  2065,  3871,     7, 17562,     2, 11991,  1048,    24,     4,
           885,     1, 11025,     7, 10187,    20,     6,  5517,     3,    -1,
            -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,
            -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,
            -1],
        [23259,    11,     6, 17457,  1195, 12240,     6,  9439, 13543,     1,
          2814,  2657,     7,  6523,  4490,    22, 15031,    82,  1383,    21,
         21972,    22,     2, 14020, 23182,    11,     1,  4490,   907,     2,
            56,    72,  1076,     1,  2196,     5,  2173, 10377,     3,    -1,
            -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,
            -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,
            -1],
        [ 3891,  

In [ ]:
torch.sum(torch.Tensor([17564, 24659, 29258, 26399,    13, 16230,  1350,  1321,     4,     6,
          1649,    20,  1471,     7,   386, 10675,     6,   784,   648,     8,
          1734,    58,     1,   351,  4033,     7,   315,   217,  4224,  2494,
            13,    16,  2880,     5,   363,   105,     3,    -1,    -1,    -1,
            -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,
            -1,    -1,    -1,    -1,    -1,    -1])!= -1)

tensor(37)